In [1]:
# !pip install -U -r /Users/ryanheuser/github/ppa-django/requirements.txt
# !pip install -r ../requirements.txt
# !pip install git+https://github.com/Princeton-CDH/parasolr
import sys; sys.path.insert(0,'..')
from ppanlp import *
here = '/Users/ryanheuser/github/ppa-nlp/notebooks'
# os.chdir(here)

In [2]:
# !pip install debug_toolbar

In [3]:
import django
PROJECTPATH = '/Users/ryanheuser/github/ppa-django'
sys.path.insert(0, PROJECTPATH)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "ppa.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
os.chdir(PROJECTPATH)
django.setup()
os.chdir(here)

In [5]:
#### ready
from ppa.archive.models import DigitizedWork
from ppa.archive.hathi import HathiObject

In [6]:
# Get data
works_with_hathi=[t for t in DigitizedWork.objects.all() if t.hathi]
len(works_with_hathi)

INFO:parasolr.django.solrclient:Connecting to default Solr http://localhost:8983/solr/ppa


5129

In [7]:
hathi_work = works_with_hathi[0]
hathi_work, hathi_work.hathi

(<DigitizedWork: mdp.39015043572422 (12-33)>,
 <ppa.archive.hathi.HathiObject at 0x12905ad60>)

In [13]:
# Adapted from hathi_page_index_data

def iter_pages(self, page_span=None, **meta):
    """Get page content for the specified digitized work from Hathi
    pairtree and return data to be indexed in solr."""

    from zipfile import ZipFile
    from eulxml.xmlmap import load_xmlobject_from_file
    from ppa.archive.hathi import MinimalMETS
    from pairtree import storage_exceptions


    # load mets record to pull metadata about the images
    try:
        mmets = load_xmlobject_from_file(self.metsfile_path(), MinimalMETS)
    except storage_exceptions.ObjectNotFoundException:
        # logger.debug(
        #     "Pairtree data for %s not found",
        #     self.hathi_id,
        # )
        return

    # read zipfile contents in place, without unzipping
    with ZipFile(self.zipfile_path()) as ht_zip:
        # yield a generator of index data for each page; iterate
        # over pages in METS structmap
        for i, page in enumerate(mmets.structmap_pages, 1):
            # if the document has a page range defined, skip any pages not in range
            if page_span and i not in page_span: continue

            # zipfile spec uses / for path regardless of OS
            pagefilename = f'{self.content_dir}/{page.text_file_location}'

            with ht_zip.open(pagefilename) as pagefile:
                try:
                    yield {
                        "source_id": self.hathi_id,
                        "page_id": page.text_file.sequence,                        
                        "page_i": i,
                        "content": pagefile.read().decode("utf-8"),
                        "order": page.order,
                        "label": page.display_label,
                        "tags": page.label.split(", ") if page.label else [],
                        "item_type": "page",
                    }
                except StopIteration:
                    return

                
HathiObject.iter_pages = iter_pages

In [18]:
f=hathi_work.hathi.iter_pages()
f

<generator object iter_pages at 0x12aaa8ac0>

In [20]:
# !pip install jsonlines
# import jsonlines

In [21]:
def save_hathi_pages_as_jsonl():
    with jsonlines.open('../data/data.hathi_pages.jsonl', mode='w') as writer:
        for text in tqdm(works_with_hathi,position=0):
            for page_d in text.hathi.iter_pages():
                writer.write(page_d)

In [ ]:
save_hathi_pages_as_jsonl()

 22%|████████▋                              | 1148/5129 [12:14<16:47,  3.95it/s]